# 第4回講義 演習

## 課題1. ロジスティック回帰の実装と学習

In [2]:
from sklearn.utils import shuffle

import numpy as np

np.random.seed(1234)

### 1. シグモイド関数とその微分

In [3]:
def sigmoid(x):
    return 1.0/(1.0+np.exp(-x))

def deriv_sigmoid(x):
    return sigmoid(x)*(1-sigmoid(x))

In [4]:
print(sigmoid (2))
print(deriv_sigmoid(2))

0.880797077978
0.104993585404


### 2. データセットの設定と重みの定義

In [5]:
# OR
train_X = np.array([[0, 1], [1, 0], [0, 0], [1, 1]])
train_y = np.array([[1], [1], [0], [1]])
test_X, test_y = train_X, train_y

# weights
W = np.random.uniform(low=-0.08, high=0.08, size=(2, 1)).astype('float32')
b = np.zeros(1).astype('float32')

### 3. train関数とtest関数

##### 誤差関数
* 負の対数尤度関数 (Negative Loglikelihood Function）
* 交差エントロピー誤差関数ともいう

$$ E ( {\bf \theta} ) =  -\sum^N_{i=1} \left[ t_i \log y ({\bf x}_i ; {\bf \theta}) + (1 - t_i) \log \{ 1 - y ({\bf x}_i ; {\bf \theta}) \}\right] $$

<ul>
<li><span class="MathJax_Preview" style="color: inherit;"></span><span class="MathJax" id="MathJax-Element-3-Frame" tabindex="0" data-mathml="<math xmlns=&quot;http://www.w3.org/1998/Math/MathML&quot;><mrow class=&quot;MJX-TeXAtom-ORD&quot;><mrow class=&quot;MJX-TeXAtom-ORD&quot;><mover><mi>y</mi><mo stretchy=&quot;false&quot;>&amp;#x005E;</mo></mover></mrow></mrow></math>" role="presentation" style="position: relative;"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-27" role="math" style="width: 0.558em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.447em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.336em 1000.45em 2.614em -999.997em); top: -2.219em; left: 0.003em;"><span class="mrow" id="MathJax-Span-28"><span class="texatom" id="MathJax-Span-29"><span class="mrow" id="MathJax-Span-30"><span class="texatom" id="MathJax-Span-31"><span class="mrow" id="MathJax-Span-32"><span class="munderover" id="MathJax-Span-33"><span style="display: inline-block; position: relative; width: 0.447em; height: 0px;"><span style="position: absolute; clip: rect(3.392em 1000.45em 4.392em -999.997em); top: -3.997em; left: 0.003em;"><span class="mi" id="MathJax-Span-34" style="font-family: STIXGeneral-Italic;">y</span><span style="display: inline-block; width: 0px; height: 4.003em;"></span></span><span style="position: absolute; top: -4.053em; left: 0.058em;"><span style="height: 0.003em; vertical-align: 0.003em; width: 0.392em; display: inline-block; overflow: hidden;"></span><span class="mo" id="MathJax-Span-35" style="font-family: STIXGeneral-Regular;">̂&nbsp;<span style="height: 0.003em; vertical-align: 0.003em; margin-left: -0.275em;"></span></span><span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span><span style="display: inline-block; width: 0px; height: 4.003em;"></span></span></span></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.225em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.33em; border-left: 0px solid; width: 0px; height: 1.27em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mrow class="MJX-TeXAtom-ORD"><mrow class="MJX-TeXAtom-ORD"><mover><mi>y</mi><mo stretchy="false">^</mo></mover></mrow></mrow></math></span></span><script type="math/tex" id="MathJax-Element-3">{ \hat{y} }</script> : 真のラベル <span class="MathJax_Preview" style="color: inherit;"></span><span class="MathJax" id="MathJax-Element-4-Frame" tabindex="0" data-mathml="<math xmlns=&quot;http://www.w3.org/1998/Math/MathML&quot;><mrow class=&quot;MJX-TeXAtom-ORD&quot;><mi>y</mi></mrow></math>" role="presentation" style="position: relative;"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-36" role="math" style="width: 0.558em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.447em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.892em 1000.45em 2.892em -999.997em); top: -2.497em; left: 0.003em;"><span class="mrow" id="MathJax-Span-37"><span class="texatom" id="MathJax-Span-38"><span class="mrow" id="MathJax-Span-39"><span class="mi" id="MathJax-Span-40" style="font-family: STIXGeneral-Italic;">y</span></span></span></span><span style="display: inline-block; width: 0px; height: 2.503em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.33em; border-left: 0px solid; width: 0px; height: 0.937em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mrow class="MJX-TeXAtom-ORD"><mi>y</mi></mrow></math></span></span><script type="math/tex" id="MathJax-Element-4">{ y }</script> の予測値ベクトル。次元は  (入力データ数, 1)</li>
<li><span class="MathJax_Preview" style="color: inherit;"></span><span class="MathJax" id="MathJax-Element-5-Frame" tabindex="0" data-mathml="<math xmlns=&quot;http://www.w3.org/1998/Math/MathML&quot;><mrow class=&quot;MJX-TeXAtom-ORD&quot;><mi>&amp;#x03C3;</mi></mrow></math>" role="presentation" style="position: relative;"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-41" role="math" style="width: 0.669em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.558em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.892em 1000.56em 2.669em -999.997em); top: -2.497em; left: 0.003em;"><span class="mrow" id="MathJax-Span-42"><span class="texatom" id="MathJax-Span-43"><span class="mrow" id="MathJax-Span-44"><span class="mi" id="MathJax-Span-45" style="font-family: STIXGeneral-Italic;">σ<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.058em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.503em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.063em; border-left: 0px solid; width: 0px; height: 0.67em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mrow class="MJX-TeXAtom-ORD"><mi>σ</mi></mrow></math></span></span><script type="math/tex" id="MathJax-Element-5">{ \sigma }</script> : <a class="keyword" href="http://d.hatena.ne.jp/keyword/%A5%B7%A5%B0%A5%E2%A5%A4%A5%C9%B4%D8%BF%F4">シグモイド関数</a>。計算結果を (0, 1) 区間に<a class="keyword" href="http://d.hatena.ne.jp/keyword/%BC%CC%C1%FC">写像</a>する</li>
<li><span class="MathJax_Preview" style="color: inherit;"></span><span class="MathJax" id="MathJax-Element-6-Frame" tabindex="0" data-mathml="<math xmlns=&quot;http://www.w3.org/1998/Math/MathML&quot;><mrow class=&quot;MJX-TeXAtom-ORD&quot;><mi>x</mi></mrow></math>" role="presentation" style="position: relative;"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-46" role="math" style="width: 0.558em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.447em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.892em 1000.45em 2.669em -999.997em); top: -2.497em; left: 0.003em;"><span class="mrow" id="MathJax-Span-47"><span class="texatom" id="MathJax-Span-48"><span class="mrow" id="MathJax-Span-49"><span class="mi" id="MathJax-Span-50" style="font-family: STIXGeneral-Italic;">x<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.503em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.063em; border-left: 0px solid; width: 0px; height: 0.67em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mrow class="MJX-TeXAtom-ORD"><mi>x</mi></mrow></math></span></span><script type="math/tex" id="MathJax-Element-6">{ x }</script> : 入力データ。次元は (入力データ数, 説明変数の数)</li>
<li><span class="MathJax_Preview" style="color: inherit;"></span><span class="MathJax" id="MathJax-Element-7-Frame" tabindex="0" data-mathml="<math xmlns=&quot;http://www.w3.org/1998/Math/MathML&quot;><mrow class=&quot;MJX-TeXAtom-ORD&quot;><mi>w</mi></mrow></math>" role="presentation" style="position: relative;"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-51" role="math" style="width: 0.836em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.669em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.892em 1000.67em 2.669em -999.997em); top: -2.497em; left: 0.003em;"><span class="mrow" id="MathJax-Span-52"><span class="texatom" id="MathJax-Span-53"><span class="mrow" id="MathJax-Span-54"><span class="mi" id="MathJax-Span-55" style="font-family: STIXGeneral-Italic;">w</span></span></span></span><span style="display: inline-block; width: 0px; height: 2.503em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.063em; border-left: 0px solid; width: 0px; height: 0.67em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mrow class="MJX-TeXAtom-ORD"><mi>w</mi></mrow></math></span></span><script type="math/tex" id="MathJax-Element-7">{ w }</script> : 係数ベクトル。次元は(クラス数 = 2)</li>
<li><span class="MathJax_Preview" style="color: inherit;"></span><span class="MathJax" id="MathJax-Element-8-Frame" tabindex="0" data-mathml="<math xmlns=&quot;http://www.w3.org/1998/Math/MathML&quot;><mrow class=&quot;MJX-TeXAtom-ORD&quot;><mi>b</mi></mrow></math>" role="presentation" style="position: relative;"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-56" role="math" style="width: 0.614em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.503em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.669em 1000.5em 2.669em -999.997em); top: -2.497em; left: 0.003em;"><span class="mrow" id="MathJax-Span-57"><span class="texatom" id="MathJax-Span-58"><span class="mrow" id="MathJax-Span-59"><span class="mi" id="MathJax-Span-60" style="font-family: STIXGeneral-Italic;">b</span></span></span></span><span style="display: inline-block; width: 0px; height: 2.503em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.063em; border-left: 0px solid; width: 0px; height: 0.937em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mrow class="MJX-TeXAtom-ORD"><mi>b</mi></mrow></math></span></span><script type="math/tex" id="MathJax-Element-8">{ b }</script> : バイアス (<a class="keyword" href="http://d.hatena.ne.jp/keyword/%A5%B9%A5%AB%A5%E9%A1%BC">スカラー</a>)</li>
</ul>

In [16]:
def train(x, t, eps=1.0):
    global W, b # to access variables that defined outside of this function.
    
    # Forward Propagation
    # print(x.shape)
    y = sigmoid(np.dot(x, W) + b)

    # Back Propagation (Cost Function: Negative Loglikelihood)
    cost = np.sum(-t*np.log(y)-(1-t)*np.log(1-y))
    delta = y - t
    #print(delta)
    # delta is single value
    
    # Update Parameters
    print(delta)
    dW = x.T* delta
    print(dW)
    db = delta
    W = W - eps*dW
    b = b - eps*db

    return cost

def test(x, t):
    # Test Cost
    y = sigmoid(np.dot(x, W) + b)
    cost = np.sum(-t*np.log(y)-(1-t)*np.log(1-y))
    return cost, y

### 4. 学習

In [17]:
# OR
train_X = np.array([[0, 1], [1, 0], [0, 0], [1, 1]])
train_y = np.array([[1], [1], [0], [1]])
test_X, test_y = train_X, train_y

# weights
W = np.random.uniform(low=-0.08, high=0.08, size=(2, 1)).astype('float32')
b = np.zeros(1).astype('float32')

In [18]:
# Epoch
for epoch in range(1000):
    # Online Learning
    for x, y in zip(train_X, train_y):
        cost = train(x[np.newaxis, :], y[np.newaxis, :])
    cost, pred_y = test(test_X, test_y)

print(pred_y)

[[-0.49987667]]
[[-0.        ]
 [-0.49987667]]
[[-0.37527134]]
[[-0.37527134]
 [-0.        ]]
[[ 0.70581576]]
[[ 0.]
 [ 0.]]
[[-0.25831115]]
[[-0.25831115]
 [-0.25831115]]
[[-0.23391694]]
[[-0.        ]
 [-0.23391694]]
[[-0.21333577]]
[[-0.21333577]
 [-0.        ]]
[[ 0.70576345]]
[[ 0.]
 [ 0.]]
[[-0.11728036]]
[[-0.11728036]
 [-0.11728036]]
[[-0.19840527]]
[[-0.        ]
 [-0.19840527]]
[[-0.18864966]]
[[-0.18864966]
 [-0.        ]]
[[ 0.66227925]]
[[ 0.]
 [ 0.]]
[[-0.07712175]]
[[-0.07712175]
 [-0.07712175]]
[[-0.18637848]]
[[-0.        ]
 [-0.18637848]]
[[-0.18028019]]
[[-0.18028019]
 [-0.        ]]
[[ 0.61182003]]
[[ 0.]
 [ 0.]]
[[-0.05546322]]
[[-0.05546322]
 [-0.05546322]]
[[-0.17860031]]
[[-0.        ]
 [-0.17860031]]
[[-0.17469413]]
[[-0.17469413]
 [-0.        ]]
[[ 0.56264576]]
[[ 0.]
 [ 0.]]
[[-0.04127658]]
[[-0.04127658]
 [-0.04127658]]
[[-0.17112692]]
[[-0.        ]
 [-0.17112692]]
[[-0.16894258]]
[[-0.16894258]
 [-0.        ]]
[[ 0.51764443]]
[[ 0.]
 [ 0.]]
[[-0.03132139]]

 [-0.00012458]]
[[-0.03480752]]
[[-0.        ]
 [-0.03480752]]
[[-0.03571156]]
[[-0.03571156]
 [-0.        ]]
[[ 0.08812798]]
[[ 0.]
 [ 0.]]
[[-0.00011811]]
[[-0.00011811]
 [-0.00011811]]
[[-0.03422651]]
[[-0.        ]
 [-0.03422651]]
[[-0.03510506]]
[[-0.03510506]
 [-0.        ]]
[[ 0.08663847]]
[[ 0.]
 [ 0.]]
[[-0.00011209]]
[[-0.00011209]
 [-0.00011209]]
[[-0.03366432]]
[[-0.        ]
 [-0.03366432]]
[[-0.03451841]]
[[-0.03451841]
 [-0.        ]]
[[ 0.08519786]]
[[ 0.]
 [ 0.]]
[[-0.00010646]]
[[-0.00010646]
 [-0.00010646]]
[[-0.03312007]]
[[-0.        ]
 [-0.03312007]]
[[-0.03395068]]
[[-0.03395068]
 [-0.        ]]
[[ 0.0838038]]
[[ 0.]
 [ 0.]]
[[-0.0001012]]
[[-0.0001012]
 [-0.0001012]]
[[-0.03259292]]
[[-0.        ]
 [-0.03259292]]
[[-0.03340097]]
[[-0.03340097]
 [-0.        ]]
[[ 0.0824541]]
[[ 0.]
 [ 0.]]
[[ -9.62833651e-05]]
[[ -9.62833651e-05]
 [ -9.62833651e-05]]
[[-0.03208209]]
[[-0.        ]
 [-0.03208209]]
[[-0.03286844]]
[[-0.03286844]
 [-0.        ]]
[[ 0.08114668]]
[[ 0

 [ -1.57233822e-05]]
[[-0.017935]]
[[-0.      ]
 [-0.017935]]
[[-0.01821123]]
[[-0.01821123]
 [-0.        ]]
[[ 0.04513377]]
[[ 0.]
 [ 0.]]
[[ -1.53013870e-05]]
[[ -1.53013870e-05]
 [ -1.53013870e-05]]
[[-0.01777755]]
[[-0.        ]
 [-0.01777755]]
[[-0.01804929]]
[[-0.01804929]
 [-0.        ]]
[[ 0.04473502]]
[[ 0.]
 [ 0.]]
[[ -1.48943109e-05]]
[[ -1.48943109e-05]
 [ -1.48943109e-05]]
[[-0.01762282]]
[[-0.        ]
 [-0.01762282]]
[[-0.01789018]]
[[-0.01789018]
 [-0.        ]]
[[ 0.04434321]]
[[ 0.]
 [ 0.]]
[[ -1.45015020e-05]]
[[ -1.45015020e-05]
 [ -1.45015020e-05]]
[[-0.01747075]]
[[-0.        ]
 [-0.01747075]]
[[-0.01773384]]
[[-0.01773384]
 [-0.        ]]
[[ 0.04395816]]
[[ 0.]
 [ 0.]]
[[ -1.41223422e-05]]
[[ -1.41223422e-05]
 [ -1.41223422e-05]]
[[-0.01732126]]
[[-0.        ]
 [-0.01732126]]
[[-0.01758018]]
[[-0.01758018]
 [-0.        ]]
[[ 0.0435797]]
[[ 0.]
 [ 0.]]
[[ -1.37562453e-05]]
[[ -1.37562453e-05]
 [ -1.37562453e-05]]
[[-0.01717429]]
[[-0.        ]
 [-0.01717429]]
[[-0

 [ 0.]]
[[ -4.23847182e-06]]
[[ -4.23847182e-06]
 [ -4.23847182e-06]]
[[-0.01170148]]
[[-0.        ]
 [-0.01170148]]
[[-0.01182507]]
[[-0.01182507]
 [-0.        ]]
[[ 0.02938107]]
[[ 0.]
 [ 0.]]
[[ -4.16417481e-06]]
[[ -4.16417481e-06]
 [ -4.16417481e-06]]
[[-0.01163395]]
[[-0.        ]
 [-0.01163395]]
[[-0.01175619]]
[[-0.01175619]
 [-0.        ]]
[[ 0.02921083]]
[[ 0.]
 [ 0.]]
[[ -4.09160063e-06]]
[[ -4.09160063e-06]
 [ -4.09160063e-06]]
[[-0.01156721]]
[[-0.        ]
 [-0.01156721]]
[[-0.01168811]]
[[-0.01168811]
 [-0.        ]]
[[ 0.02904253]]
[[ 0.]
 [ 0.]]
[[ -4.02069970e-06]]
[[ -4.02069970e-06]
 [ -4.02069970e-06]]
[[-0.01150122]]
[[-0.        ]
 [-0.01150122]]
[[-0.01162081]]
[[-0.01162081]
 [-0.        ]]
[[ 0.02887615]]
[[ 0.]
 [ 0.]]
[[ -3.95142416e-06]]
[[ -3.95142416e-06]
 [ -3.95142416e-06]]
[[-0.01143597]]
[[-0.        ]
 [-0.01143597]]
[[-0.01155427]]
[[-0.01155427]
 [-0.        ]]
[[ 0.02871167]]
[[ 0.]
 [ 0.]]
[[ -3.88372777e-06]]
[[ -3.88372777e-06]
 [ -3.88372777e-

[[ -1.68092939e-06]]
[[ -1.68092939e-06]
 [ -1.68092939e-06]]
[[-0.00863938]]
[[-0.        ]
 [-0.00863938]]
[[-0.00870843]]
[[-0.00870843]
 [-0.        ]]
[[ 0.02166838]]
[[ 0.]
 [ 0.]]
[[ -1.65916520e-06]]
[[ -1.65916520e-06]
 [ -1.65916520e-06]]
[[-0.00860244]]
[[-0.        ]
 [-0.00860244]]
[[-0.00867093]]
[[-0.00867093]
 [-0.        ]]
[[ 0.02157544]]
[[ 0.]
 [ 0.]]
[[ -1.63777454e-06]]
[[ -1.63777454e-06]
 [ -1.63777454e-06]]
[[-0.00856582]]
[[-0.        ]
 [-0.00856582]]
[[-0.00863374]]
[[-0.00863374]
 [-0.        ]]
[[ 0.0214833]]
[[ 0.]
 [ 0.]]
[[ -1.61674947e-06]]
[[ -1.61674947e-06]
 [ -1.61674947e-06]]
[[-0.0085295]]
[[-0.       ]
 [-0.0085295]]
[[-0.00859687]]
[[-0.00859687]
 [-0.        ]]
[[ 0.02139193]]
[[ 0.]
 [ 0.]]
[[ -1.59608220e-06]]
[[ -1.59608220e-06]
 [ -1.59608220e-06]]
[[-0.00849349]]
[[-0.        ]
 [-0.00849349]]
[[-0.00856031]]
[[-0.00856031]
 [-0.        ]]
[[ 0.02130134]]
[[ 0.]
 [ 0.]]
[[ -1.57576516e-06]]
[[ -1.57576516e-06]
 [ -1.57576516e-06]]
[[-0.00

...

 [-0.        ]]
[[ 0.00649291]]
[[ 0.]
 [ 0.]]
[[ -4.40492586e-08]]
[[ -4.40492586e-08]
 [ -4.40492586e-08]]
[[-0.00259123]]
[[-0.        ]
 [-0.00259123]]
[[-0.00259777]]
[[-0.00259777]
 [-0.        ]]
[[ 0.00648451]]
[[ 0.]
 [ 0.]]
[[ -4.38780802e-08]]
[[ -4.38780802e-08]
 [ -4.38780802e-08]]
[[-0.00258789]]
[[-0.        ]
 [-0.00258789]]
[[-0.00259441]]
[[-0.00259441]
 [-0.        ]]
[[ 0.00647612]]
[[ 0.]
 [ 0.]]
[[ -4.37077873e-08]]
[[ -4.37077873e-08]
 [ -4.37077873e-08]]
[[-0.00258455]]
[[-0.        ]
 [-0.00258455]]
[[-0.00259105]]
[[-0.00259105]
 [-0.        ]]
[[ 0.00646776]]
[[ 0.]
 [ 0.]]
[[ -4.35383739e-08]]
[[ -4.35383739e-08]
 [ -4.35383739e-08]]
[[-0.00258122]]
[[-0.        ]
 [-0.00258122]]
[[-0.00258771]]
[[-0.00258771]
 [-0.        ]]
[[ 0.00645942]]
[[ 0.]
 [ 0.]]
[[ -4.33698347e-08]]
[[ -4.33698347e-08]
 [ -4.33698347e-08]]
[[-0.0025779]]
[[-0.       ]
 [-0.0025779]]
[[-0.00258437]]
[[-0.00258437]
 [-0.        ]]
[[ 0.0064511]]
[[ 0.]
 [ 0.]]
[[ -4.32021637e-08]]
[

[[-0.00244259]]
[[-0.00244259]
 [-0.        ]]
[[ 0.00609769]]
[[ 0.]
 [ 0.]]
[[ -3.64723340e-08]]
[[ -3.64723340e-08]
 [ -3.64723340e-08]]
[[-0.00243384]]
[[-0.        ]
 [-0.00243384]]
[[-0.00243962]]
[[-0.00243962]
 [-0.        ]]
[[ 0.00609027]]
[[ 0.]
 [ 0.]]
[[ -3.63392074e-08]]
[[ -3.63392074e-08]
 [ -3.63392074e-08]]
[[-0.00243089]]
[[-0.        ]
 [-0.00243089]]
[[-0.00243665]]
[[-0.00243665]
 [-0.        ]]
[[ 0.00608288]]
[[ 0.]
 [ 0.]]
[[ -3.62067274e-08]]
[[ -3.62067274e-08]
 [ -3.62067274e-08]]
[[-0.00242794]]
[[-0.        ]
 [-0.00242794]]
[[-0.00243369]]
[[-0.00243369]
 [-0.        ]]
[[ 0.0060755]]
[[ 0.]
 [ 0.]]
[[ -3.60748902e-08]]
[[ -3.60748902e-08]
 [ -3.60748902e-08]]
[[-0.002425]]
[[-0.      ]
 [-0.002425]]
[[-0.00243074]]
[[-0.00243074]
 [-0.        ]]
[[ 0.00606814]]
[[ 0.]
 [ 0.]]
[[ -3.59436922e-08]]
[[ -3.59436922e-08]
 [ -3.59436922e-08]]
[[-0.00242207]]
[[-0.        ]
 [-0.00242207]]
[[-0.00242779]]
[[-0.00242779]
 [-0.        ]]
[[ 0.00606079]]
[[ 0.]
 [

 [-0.        ]]
[[ 0.00566328]]
[[ 0.]
 [ 0.]]
[[ -2.92082769e-08]]
[[ -2.92082769e-08]
 [ -2.92082769e-08]]
[[-0.0022608]]
[[-0.       ]
 [-0.0022608]]
[[-0.00226579]]
[[-0.00226579]
 [-0.        ]]
[[ 0.00565689]]
[[ 0.]
 [ 0.]]
[[ -2.91092430e-08]]
[[ -2.91092430e-08]
 [ -2.91092430e-08]]
[[-0.00225825]]
[[-0.        ]
 [-0.00225825]]
[[-0.00226323]]
[[-0.00226323]
 [-0.        ]]
[[ 0.0056505]]
[[ 0.]
 [ 0.]]
[[ -2.90106563e-08]]
[[ -2.90106563e-08]
 [ -2.90106563e-08]]
[[-0.0022557]]
[[-0.       ]
 [-0.0022557]]
[[-0.00226067]]
[[-0.00226067]
 [-0.        ]]
[[ 0.00564414]]
[[ 0.]
 [ 0.]]
[[ -2.89125139e-08]]
[[ -2.89125139e-08]
 [ -2.89125139e-08]]
[[-0.00225317]]
[[-0.        ]
 [-0.00225317]]
[[-0.00225813]]
[[-0.00225813]
 [-0.        ]]
[[ 0.00563778]]
[[ 0.]
 [ 0.]]
[[ -2.88148138e-08]]
[[ -2.88148138e-08]
 [ -2.88148138e-08]]
[[-0.00225064]]
[[-0.        ]
 [-0.00225064]]
[[-0.00225558]]
[[-0.00225558]
 [-0.        ]]
[[ 0.00563144]]
[[ 0.]
 [ 0.]]
[[ -2.87175531e-08]]
[[ -

[[ -2.45998978e-08]]
[[ -2.45998978e-08]
 [ -2.45998978e-08]]
[[-0.00213546]]
[[-0.        ]
 [-0.00213546]]
[[-0.00213992]]
[[-0.00213992]
 [-0.        ]]
[[ 0.00534303]]
[[ 0.]
 [ 0.]]
[[ -2.45211124e-08]]
[[ -2.45211124e-08]
 [ -2.45211124e-08]]
[[-0.00213319]]
[[-0.        ]
 [-0.00213319]]
[[-0.00213764]]
[[-0.00213764]
 [-0.        ]]
[[ 0.00533733]]
[[ 0.]
 [ 0.]]
[[ -2.44426629e-08]]
[[ -2.44426629e-08]
 [ -2.44426629e-08]]
[[-0.00213092]]
[[-0.        ]
 [-0.00213092]]
[[-0.00213536]]
[[-0.00213536]
 [-0.        ]]
[[ 0.00533165]]
[[ 0.]
 [ 0.]]
[[ -2.43645474e-08]]
[[ -2.43645474e-08]
 [ -2.43645474e-08]]
[[-0.00212865]]
[[-0.        ]
 [-0.00212865]]
[[-0.00213308]]
[[-0.00213308]
 [-0.        ]]
[[ 0.00532598]]
[[ 0.]
 [ 0.]]
[[ -2.42867645e-08]]
[[ -2.42867645e-08]
 [ -2.42867645e-08]]
[[-0.00212639]]
[[-0.        ]
 [-0.00212639]]
[[-0.00213082]]
[[-0.00213082]
 [-0.        ]]
[[ 0.00532032]]
[[ 0.]
 [ 0.]]
[[ -2.42093122e-08]]
[[ -2.42093122e-08]
 [ -2.42093122e-08]]
[[-

 [ 0.]]
[[ -2.12330569e-08]]
[[ -2.12330569e-08]
 [ -2.12330569e-08]]
[[-0.00203356]]
[[-0.        ]
 [-0.00203356]]
[[-0.00203761]]
[[-0.00203761]
 [-0.        ]]
[[ 0.00508786]]
[[ 0.]
 [ 0.]]
[[ -2.11682989e-08]]
[[ -2.11682989e-08]
 [ -2.11682989e-08]]
[[-0.0020315]]
[[-0.       ]
 [-0.0020315]]
[[-0.00203554]]
[[-0.00203554]
 [-0.        ]]
[[ 0.0050827]]
[[ 0.]
 [ 0.]]
[[ -2.11038038e-08]]
[[ -2.11038038e-08]
 [ -2.11038038e-08]]
[[-0.00202944]]
[[-0.        ]
 [-0.00202944]]
[[-0.00203347]]
[[-0.00203347]
 [-0.        ]]
[[ 0.00507754]]
[[ 0.]
 [ 0.]]
[[ -2.10395703e-08]]
[[ -2.10395703e-08]
 [ -2.10395703e-08]]
[[-0.00202738]]
[[-0.        ]
 [-0.00202738]]
[[-0.00203141]]
[[-0.00203141]
 [-0.        ]]
[[ 0.0050724]]
[[ 0.]
 [ 0.]]
[[ -2.09755973e-08]]
[[ -2.09755973e-08]
 [ -2.09755973e-08]]
[[-0.00202533]]
[[-0.        ]
 [-0.00202533]]
[[-0.00202935]]
[[-0.00202935]
 [-0.        ]]
[[ 0.00506727]]
[[ 0.]
 [ 0.]]
[[ -2.09118831e-08]]
[[ -2.09118831e-08]
 [ -2.09118831e-08]]


## 課題2. 活性化関数とその微分の実装

###  1. シグモイド関数とその微分

In [9]:
def sigmoid(x):
    return 1.0/(1.0+np.exp(-x))

def deriv_sigmoid(x):
    return sigmoid(x)*(1-sigmoid(x))

### 2. ソフトマックス関数とその微分

In [10]:
def softmax(x):
    return np.exp(x) / np.sum(np.exp(x))
    

def deriv_softmax(x):
    return softmax(x)*(1 - softmax(x))

### 3. tanh関数とその微分

In [11]:
def tanh(x):
    return np.tanh(x)


def deriv_tanh(x):
    return 1 - tanh(x)**2

## 課題3. 多層パーセプトロンの実装と学習

### 1. データセットの設定と重みの定義

In [12]:
# XOR
train_X = np.array([[0, 1], [1, 0], [0, 0], [1, 1]])
train_y = np.array([[1], [1], [0], [0]])
test_X, test_y = train_X, train_y

# Layer1 weights
W1 = np.random.uniform(low=-0.08, high=0.08, size=(2, 3)).astype('float32')
b1 = np.zeros(3).astype('float32')

# Layer2 weights
W2 = np.random.uniform(low=-0.08, high=0.08, size=(3, 1)).astype('float32')
b2 = np.zeros(1).astype('float32')

### 2.train関数とtest関数

##### 誤差関数
* 負の対数尤度関数 (Negative Loglikelihood Function）
* 交差エントロピー誤差関数ともいう

$$ E ( {\bf \theta} ) =  -\sum^N_{i=1} \left[ t_i \log y ({\bf x}_i ; {\bf \theta}) + (1 - t_i) \log \{ 1 - y ({\bf x}_i ; {\bf \theta}) \}\right] $$

In [19]:
def train(x, t, eps=1.0):
    global W1, b1, W2, b2 # to access variables that defined outside of this function.
    
    # Forward Propagation Layer1
    u1 = np.dot(x, W1) + b1
    z1 = sigmoid(u1)
    
    # Forward Propagation Layer2
    u2 = np.dot(z1, W2) + b2
    z2 = sigmoid(u2)
    
    # Back Propagation (Cost Function: Negative Loglikelihood)
    y = z2
    cost = np.sum(-t*np.log(y) - (1 - t)*np.log(1 - y))
    delta_2 = y - t
    # PRML(p.245参照)
    # W_{jk}はn-1層目のj番目の要素をn層目のk番目に変換する値である
    # W.T_{kj}はn層目のk番目の値をn-1番目のj番目のノードに及ぼす値である
    delta_1 = deriv_sigmoid(u1) * np.dot(delta_2, W2.T)

    # Update Parameters Layer1
    # by x.T, create same dimension of W (broadcast)
    dW1 = np.dot(x.T, delta_1)
    db1 = np.matmul(np.ones(len(x)), delta_1)   # broadvast 
    W1 = W1 - eps*dW1
    b1 = b1 - eps*db1
    
    # Update Parameters Layer2
    dW2 = np.dot(z1.T, delta_2)
    db2 = np.dot(np.ones(len(z1)), delta_2)
    W2 = W2 - eps*dW2
    b2 = b2 - eps*db2

    return cost

def test(x, t):
    # Forward Propagation Layer1
    u1 = np.matmul(x, W1) + b1
    z1 = sigmoid(u1)
    
    # Forward Propagation Layer2
    u2 = np.matmul(z1, W2) + b2
    z2 = sigmoid(u2)
    
    y = z2
    
    # Test Cost
    cost = np.sum(-t*np.log(y)-(1-t)*np.log(1-y))
    return cost, y

### 3. 学習

In [20]:
# Epoch
for epoch in range(2000):
    # Online Learning
    for x, y in zip(train_X, train_y):
        cost = train(x[np.newaxis, :], y[np.newaxis, :])
    cost, pred_y = test(test_X, test_y)

print(pred_y)

[[  9.99780521e-01]
 [  9.99779077e-01]
 [  9.47912562e-04]
 [  9.99981753e-01]]
